In [1]:
import pandas as pd

In [2]:
files = ['artist', 'canvas_size', 'image_link', 'museum_hours', 'museum', 'product_size', 'subject', 'work']

In [3]:
for file in files:
    globals()[file] = pd.read_csv(f'./dataset/{file}.csv')

### 1.	Which canva size costs the most?

In [4]:
ps = product_size[product_size['sale_price'] == product_size['sale_price'].max()]
ps['size_id'] = ps['size_id'].astype(int)
df = pd.merge(canvas_size, ps, on = 'size_id', how = 'inner')
df[['label', 'sale_price']]

C:\Users\hamzumoh\AppData\Local\Temp\ipykernel_26576\3636273477.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ps['size_id'] = ps['size_id'].astype(int)


,label,sale_price
0,"48"" x 96""(122 cm x 244 cm)",1115


### 2. Delete duplicate records from work, product_size, subject and image_link tables

In [5]:
work.drop_duplicates()
product_size.drop_duplicates()
subject.drop_duplicates()
image_link.drop_duplicates()

,work_id,url,thumbnail_small_url,thumbnail_large_url
0,181978,https://v5.airtableusercontent.com/v1/15/15/16...,https://v5.airtableusercontent.com/v1/15/15/16...,https://v5.airtableusercontent.com/v1/15/15/16...
1,173188,https://v5.airtableusercontent.com/v1/15/15/16...,https://v5.airtableusercontent.com/v1/15/15/16...,https://v5.airtableusercontent.com/v1/15/15/16...
2,194065,https://v5.airtableusercontent.com/v1/15/15/16...,https://v5.airtableusercontent.com/v1/15/15/16...,https://v5.airtableusercontent.com/v1/15/15/16...
3,129337,https://v5.airtableusercontent.com/v1/15/15/16...,https://v5.airtableusercontent.com/v1/15/15/16...,https://v5.airtableusercontent.com/v1/15/15/16...
4,141073,https://v5.airtableusercontent.com/v1/15/15/16...,https://v5.airtableusercontent.com/v1/15/15/16...,https://v5.airtableusercontent.com/v1/15/15/16...
...,...,...,...,...
14770,194297,https://v5.airtableusercontent.com/v1/15/15/16...,https://v5.airtableusercontent.com/v1/15/15/16...,https://v5.airtableusercontent.com/v1/15/15/16...
14771,182986,https://v5.airtableusercontent.com/v1/15/15/16...,https://v5.airtableusercontent.com/v1/15/15/16...,https://v5.airtableusercontent.com/v1/15/15/16...
14772,1477,https://v5.airtableusercontent.com/v1/15/15/16...,https://v5.airtableusercontent.com/v1/15/15/16...,https://v5.airtableusercontent.com/v1/15/15/16...
14773,28147,https://v5.airtableusercontent.com/v1/15/15/16...,https://v5.airtableusercontent.com/v1/15/15/16...,https://v5.airtableusercontent.com/v1/15/15/16...


### 3. Identify the museums with invalid city information in the given dataset

In [25]:
df = None
df = museum
df[df['city'].str.contains(r'^[\d]+$', regex = True)]
# df[~df['city'].str.contains(r'^[A-Za-z\s]+$', regex = True)]

,museum_id,name,address,city,state,postal,country,phone,url
4,34,The State Hermitage Museum,Palace Square,2,Sankt-Peterburg,190000,Russia,7 812 710-90-79,https://www.hermitagemuseum.org/wps/portal/her...
6,36,Museum Folkwang,Museumsplatz 1,45128,Essen,NaN,Germany,49 201 8845000,https://www.museum-folkwang.de/en
7,37,Museum of Grenoble,5 Pl. de Lavalette,38000,Grenoble,NaN,France,33 4 76 63 44 44,https://www.museedegrenoble.fr/1986-the-museum...
8,38,Musée des Beaux-Arts de Quimper,40 Pl. Saint-Corentin,29000,Quimper,NaN,France,33 2 98 95 45 20,https://www.mbaq.fr/en/home-3.html
10,40,Musée du Louvre,Rue de Rivoli,75001,Paris,NaN,France,33 1 40 20 50 50,https://www.louvre.fr/en


### 4. Museum_Hours table has 1 invalid entry. Identify it and remove it.

In [34]:
df = None
df = museum_hours.groupby(['museum_id', 'day'])['open'].count().reset_index().query('open != 1')
df

,museum_id,day,open
307,80,Saturday,2


### 5. Fetch the top 10 most famous painting subject

In [6]:
 subject.groupby('subject', as_index = False)['work_id'].count().sort_values('work_id', ascending= False).reset_index().loc[0:9, ['subject', 'work_id']].rename(columns = {'work_id' : 'no of paintings'})

,subject,no of paintings
0,Portraits,1070
1,Nude,525
2,Landscape Art,495
3,Rivers/Lakes,480
4,Abstract/Modern Art,457
5,Flowers,457
6,Still-Life,395
7,Seascapes,324
8,Marine Art/Maritime,268
9,Horses,265


### 6. Identify the museums which are open on both Sunday and Monday. Display

In [7]:
mus_hour = museum_hours[(museum_hours['day'] == 'Sunday') | (museum_hours['day'] == 'Monday')]
day_counts = mus_hour.groupby('museum_id')['day'].count()
result = day_counts[day_counts > 1]
df = pd.merge(result, museum, on = 'museum_id', how = 'inner')
df[['name', 'city']]

,name,city
0,The Museum of Modern Art,New York
1,Pushkin State Museum of Fine Arts,Moscow
2,National Gallery of Victoria,Melbourne
3,The Metropolitan Museum of Art,New York
4,Museum of Grenoble,38000
5,Nelson-Atkins Museum of Art,Kansas City
6,Musée du Louvre,75001
7,National Maritime Museum,London
8,Museum of Fine Arts Boston,Boston
9,Rijksmuseum,Amsterdam


### 7. How many museums are open every single day?	

In [8]:
df = None
df = museum_hours.groupby('museum_id')['day'].count().reset_index().query('day == 7')
df.shape[0]

18

### 8. Which are the top 5 most popular museum? (Popularity is defined based on most no of paintings in a museum). Output museum_id, name , city , number of paintings and rank.

In [9]:
df = None
df = work.groupby('museum_id')['work_id'].count().reset_index().sort_values('work_id', ascending = False)
df['rank'] = df['work_id'].rank(method = 'min', ascending = False)
df = df[0:5]
pd.merge(museum, df, on = 'museum_id', how = 'inner')[['museum_id', 'name', 'city', 'work_id', 'rank']].rename(columns = {'work_id' : 'no of paintings'})

,museum_id,name,city,no of paintings,rank
0,35,The Metropolitan Museum of Art,New York,939,1.0
1,43,Rijksmuseum,Amsterdam,452,2.0
2,46,National Gallery of Art,Washington,375,4.0
3,47,National Gallery,London,423,3.0
4,51,The Barnes Foundation,Philadelphia,350,5.0


### 9. Who are the top 5 most popular artist? (Popularity is defined based on most no of paintings done by an artist). Output artist name , style and number of paintings.

In [10]:
df = None
df = work.groupby('artist_id')['work_id'].count().reset_index().sort_values('work_id' , ascending = False)
df['rank'] = df['work_id'].rank(method = 'min', ascending = False)
df = df[0:5]
pd.merge(artist, df, on = 'artist_id', how = 'inner')[['artist_id', 'full_name', 'style', 'work_id', 'rank']].rename(columns = {'work_id' : 'no of paintings'})

,artist_id,full_name,style,no of paintings,rank
0,500,Pierre-Auguste Renoir,Impressionist,469,1.0
1,550,Claude Monet,Impressionist,378,2.0
2,559,Albert Marquet,Post-Impressionist,233,5.0
3,649,Maurice Utrillo,Realist,253,4.0
4,677,Vincent Van Gogh,Post-Impressionist,308,3.0


### 10. Which museum is open for the longest during a day. Dispay museum name, state, day , opens , clsoe and hours open?


In [11]:
df = None
df = museum_hours
df['open'] = df['open'].astype(str)
df['open'] = df['open'].str.strip()
df['open'] = pd.to_datetime(df['open'], format='%I:%M:%p')

In [12]:
df['close'] = df['close'].astype(str)
df['close'] = df['close'].str.replace(' ','')
df['close'] = pd.to_datetime(df['close'], format='%I:%M:%p')

In [13]:
df['diff'] = df['close'] - df['open']

In [14]:
df['diff'] = df['close'] - df['open']
df = df.sort_values('diff', ascending = False)
df = df.head(1)
df = pd.merge(df, museum, on = 'museum_id', how = 'inner')

In [15]:
df[['name','state', 'day', 'open', 'close', 'diff']]

,name,state,day,open,close,diff
0,Musée du Louvre,Paris,Friday,1900-01-01 09:00:00,1900-01-01 21:45:00,0 days 12:45:00
